In [ ]:
!pip install plotly==4.1.0
!pip install ipywidgets
!npm install -g electron@1.8.4 orca

In [ ]:
import pandas as pd
import numpy as np
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import glob
import json

from plotly.subplots import make_subplots
from decimal import *
from datetime import datetime  

getcontext().prec = 4
py.io.orca.config.default_width = 1080

In [ ]:
def strToDate(row):
    return datetime.strptime(row['date'], '%Y-%m-%d').date()
def weiToGwei(row):
    return row['gasPrice'] / 10 ** 9
def gasToEth(row):
    return row['gasUsed'] * row['gasPrice']/10**9
def gasToUsd(row):
    return row['ethCost'] * row['ethUsdRate']
def getWeek(row):
    week = "{}".format(row['dateTime'].isocalendar()[1])
    return "{} w#{}".format(row['dateTime'].isocalendar()[0], week.zfill(2))
def getMonth(row):
    month = "{}".format(row['dateTime'].month)
    return "{} m#{}".format(row['dateTime'].year, month.zfill(2))    

path = './data/'
filenames = glob.glob(path + "*.csv")

dataList = []

for filename in filenames:
    df = pd.read_csv(filename, index_col=None, header=0)
    dataList.append(df)
    
pathToEthCurrency = './eth_currency.csv'
dataEthCurrency = pd.read_csv(pathToEthCurrency, index_col=None, header=0)
dataEthCurrency['date'] = dataEthCurrency.apply (lambda row: strToDate(row), axis=1)

data = pd.concat(dataList, axis=0, ignore_index=True, sort=False)
data['method'].replace(np.nan, "no_name", inplace=True)
data['dateTime'] = data['timeStamp'].apply(datetime.fromtimestamp)
data['date'] = data['dateTime'].apply(datetime.date)
data['week'] = data.apply(lambda row: getWeek(row), axis=1)
data['month'] = data.apply(lambda row: getMonth(row), axis=1)

data = data.join(dataEthCurrency.set_index('date'), on='date')
data['ethUsdRate'].replace(np.nan, dataEthCurrency['ethUsdRate'][0], inplace=True)
data['gasPrice'] = data.apply(lambda row: weiToGwei(row), axis=1)
data['ethCost'] = data.apply(lambda row: gasToEth(row), axis=1)
data['usdCost'] = data.apply(lambda row: gasToUsd(row), axis=1)

data.replace("", np.nan, inplace=True)

methods = np.sort(data.method.unique())

In [ ]:
totalCost = {}
methodCost = {}
methodPercentage = {}
ethUsdRate = {}
gasPriceByMethod = {}

# ANALYTIC PLOTS

## 1. Stacked area time series plot showing the aggregate fee costs per method, bucketed into period windows 


In [ ]:
def update_plot_1(period, currency, aggregation):
    if not (period, currency) in methodCost.keys():
        methodCost[(period, currency)] = data.groupby([period, 'method'], as_index=False).agg({currency: ['sum', 'mean']})
    if not (period, currency) in ethUsdRate.keys():
        ethUsdRate[(period, currency)] = data.groupby([period], as_index=False).agg({'ethUsdRate': ['min', 'mean', 'max']})

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    methodCostGrouped = methodCost[(period, currency)]
    # average cost per method       
    for method in methods:
        methodCostByPeriod = methodCostGrouped[methodCostGrouped.method == method]
        trace = go.Scatter(
            x=methodCostByPeriod[period],
            y=methodCostByPeriod[currency][aggregation],
            hoverinfo='x+y',
            mode='lines',
            stackgroup='one',
            name=method
        )
        fig.add_trace(trace)

    # show eth to usd rate
    if (currency == 'usdCost'):
        ethUsdRateByPeriod = ethUsdRate[(period, currency)]
        x_period = ethUsdRateByPeriod[period]
        y_mean = np.around(ethUsdRateByPeriod['ethUsdRate']['mean'].array, decimals = 3)
        y_upper = np.around(ethUsdRateByPeriod['ethUsdRate']['max'].array, decimals = 3)
        y_lower = np.around(ethUsdRateByPeriod['ethUsdRate']['min'].array, decimals = 3)
        
        fig.add_trace(
            go.Scatter(
                x=x_period,
                y=y_lower,
                mode='lines',
                line_color='rgba(255,255,255,0)',
                showlegend=False,
            ),
            secondary_y=True,
        )
        fig.add_trace(
            go.Scatter(
                x=x_period,
                y=y_upper,
                mode='lines',
                fill='tonexty',
                fillcolor='rgba(0,100,80,0.2)',
                line_color='rgba(255,255,255,0)',
                showlegend=False,
            ),
            secondary_y=True,
        )
        fig.add_trace(
            go.Scatter(
                x=x_period,
                y=y_mean,
                mode='lines',
                line_color='rgb(0,100,80)',
                name="ETH to USD rate",
            ),
            secondary_y=True,
        )
        fig.update_yaxes(title_text='ETH price, $', secondary_y=True)

    fig.update_layout(
        title='Fig.1 {} cost per method (grouped by {})'.format('Total' if aggregation == 'sum' else 'Average', period),
        xaxis=dict(title='period, {}'.format(period)),
        yaxis=dict(title='cost, {}'.format('$' if currency == 'usdCost' else 'ETH')),
        annotations=[
            dict(
                text='Method',
                x=1.10,
                y=1.10,
                showarrow=False,
                xref="paper",
                yref="paper",
            )
        ],
    )
    fig.show()
        
period = widgets.Dropdown(
    options=[('Date', 'date'), ('Week', 'week'), ('Month', 'month')],
    value='week',
    description='Period:',
)
currency = widgets.Dropdown(
    options=[('USD', 'usdCost'), ('ETH', 'ethCost')],
    value='usdCost',
    description='Currency:',
)
aggregation = widgets.Dropdown(
    options=[('Sum', 'sum'), ('Avg', 'mean')],
    value='sum',
    description='Aggregation:',
)
widgets.interactive(update_plot_1, period=period, currency=currency, aggregation=aggregation)

## 2. Stacked area time series plot showing the relative fee costs, bucketed into period windows 

In [ ]:
def update_plot_2(period, currency):
    if not (period, currency) in totalCost.keys():
        totalCost[(period, currency)] = data.groupby([period]).agg({currency: 'sum'}).to_dict()[currency]

    if not (period, currency) in methodPercentage.keys():
        methodPercentage[(period, currency)] = data.groupby([period, 'method'], as_index=False).agg({currency: 'sum'})
        methodPercentage[(period, currency)][currency] = methodPercentage[(period, currency)].apply(lambda row: row[currency] / totalCost[(period, currency)][row[period]] * 100, axis=1)

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    methodPercentageGrouped = methodPercentage[(period, currency)]
    # average cost per method       
    for method in methods:
        methodPercentageByPeriod = methodPercentageGrouped[methodPercentageGrouped.method == method]
        trace = go.Scatter(
            x=methodPercentageByPeriod[period],
            y=methodPercentageByPeriod[currency],
            hoverinfo='x+y',
            mode='lines',
            stackgroup='one',
            name=method
        )
        fig.add_trace(trace)

    fig.update_layout(
        title='Fig.2 % of total cost per method (grouped by {})'.format(period),
        xaxis=dict(title='period, {}'.format(period), type='category', categoryorder='category ascending',),
        yaxis=dict(title='% of total cost, {}'.format('$' if currency == 'usdCost' else 'ETH')),
        annotations = [
            dict(
                text='Method',
                x=1.10,
                y=1.10,
                showarrow=False,
                xref="paper",
                yref="paper",
            )
        ]
    )
    fig.show()
        
period = widgets.Dropdown(
    options=[('Date', 'date'), ('Week', 'week'), ('Month', 'month')],
    value='week',
    description='Period:',
)
currency = widgets.Dropdown(
    options=[('USD', 'usdCost'), ('ETH', 'ethCost')],
    value='usdCost',
    description='Currency:',
)
widgets.interactive(update_plot_2, period=period, currency=currency)

## 3. Time series plot showing just the gas price over time.


In [ ]:
def update_plot_3(period):
    if not (period) in gasPriceByMethod.keys():
        gasPriceByMethod[(period)] = data.groupby([period, 'method'], as_index=False).agg({'gasPrice': 'mean'})

    fig = make_subplots()

    # average per method
    for method in methods:
        gasPriceByMethodByPeriod = gasPriceByMethod[(period)].query("method == '{}'".format(method))    
        trace = go.Scatter(
            x=gasPriceByMethodByPeriod[period],
            y=gasPriceByMethodByPeriod['gasPrice'],
            name='{} gas price'.format(method),
            mode='lines+markers'
        )
        fig.add_trace(trace)

    fig.update_layout(
        title='Fig.3 Average gas price (grouped by {})'.format(period),
        xaxis=dict(title='period, {}'.format(period)),
        yaxis=dict(title='gas price, Gwei'),
        annotations=[
            dict(
                text='Method',
                x=1.10,
                y=1.10,
                showarrow=False,
                xref="paper",
                yref="paper",
            )
        ],
        xaxis_rangeslider_visible=True,
    )
    fig.show()
    
period = widgets.Dropdown(
    options=[('Date', 'date'), ('Week', 'week'), ('Month', 'month')],
    value='date',
    description='Period:',
)
widgets.interactive(update_plot_3, period=period)